In [2]:
import pandas as pd
import numpy as np
import os

In [101]:
xls = pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung.xlsx')

In [102]:
def mark_true_false(df: pd.DataFrame, column:str, symbol: str) -> pd.DataFrame:
    """
    Change the value of a column to True if it is equal to a symbol, False otherwise
    """
    df[column] = df[column].apply(lambda x: True if x == symbol else False)
    return df

In [103]:
def replace_null_zero(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """
    Replace null values in a column by 0
    """
    df[columns] = df[columns].fillna(0)
    return df

### Gross Demand

In [276]:
df = pd.read_excel(xls, 'Gross Demand')

In [277]:
df.head()

,DC (Gross Demand after set & network drilldown) all DC,Feb.24,Mar.24,Apr.24,May.24,Jun.24,Jul.24,Aug.24,Sep.24,Oct.24,...,Dec.24,Jan.25,Feb.25,Mar.25,Apr.25,May.25,Jun.25,Jul.25,TOTAL,Unnamed: 20
0,- 92219900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,0011,5950.370117,1939.369995,1603.949951,3684.739990,5094.779785,5665.259766,5090.470215,7445.919922,7722.930176,...,8844.110352,9379.929688,11907.580078,17273.619141,15001.759766,13034.870117,15063.980469,NaN,143424.939331,
2,0058,893.669983,1592.520020,1287.579956,3325.090088,4598.339844,5080.549805,4472.859863,5071.279785,5135.580078,...,4919.759766,4743.169922,5059.560059,5598.680176,4995.270020,4595.290039,4687.979980,NaN,71691.829285,
3,0099,764.919983,118.080002,682.520020,689.770020,828.570007,1278.459961,1016.679993,1917.260010,1797.689941,...,2048.489990,2346.100098,2424.800049,2362.399902,2521.679932,2318.429932,2355.379883,NaN,27388.429672,
4,0047,2.580000,4.170000,6.150000,5.070000,4.590000,5.730000,4.610000,5.000000,6.240000,...,5.430000,17.879999,5.810000,6.250000,4.820000,5.100000,5.240000,NaN,99.459999,


In [122]:
df.rename(columns={'DC (Gross Demand after set & network drilldown) all DC': 'Werk'}, inplace=True)

In [123]:
df.drop(['Unnamed: 20', 'TOTAL'], axis=1, inplace=True)

In [124]:
df['Material'] = df[df.drop(['Werk'], axis=1).isnull().all(axis=1)]['Werk']
df['Material'] = df['Material'].fillna(method='ffill')
df = df[df['Werk'] != df['Material']]

C:\Users\TimKnudsen\AppData\Local\Temp\ipykernel_6688\3695123119.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Material'] = df['Material'].fillna(method='ffill')


In [125]:
# Clear lines where 'Total' appears in the Werk column
df = df[df['Werk'].str.contains('Total') == False]
df = df[df['Werk'].str.contains('TOTAL') == False]

In [126]:
df.fillna(0, inplace=True)

In [127]:
df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='Gross Demand')

In [134]:
df['Werk'] = df['Werk'].astype(int)
# Cut off the first character of the Material column
df['Material'] = df['Material'].str[1:]

In [136]:
df.head(10)

,Material,Werk,Month,Gross Demand
0,92219900,11,Feb.24,5950.370117
1,92219900,58,Feb.24,893.669983
2,92219900,99,Feb.24,764.919983
3,92219900,47,Feb.24,2.580000
4,92219900,42,Feb.24,4.000000
5,92219900,28,Feb.24,0.020000
6,61630001,42,Feb.24,335.149994
7,61630001,11,Feb.24,120.779999
8,61630001,49,Feb.24,39.139999
9,61630001,28,Feb.24,9.110000


In [137]:
# Export the dataframe to a csv file
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\Gross_Demand.csv', index=False)

### EKPO

In [138]:
df = pd.read_excel(xls, 'EKPO')

In [140]:
df = mark_true_false(df, 'ELi', 'X')
df = mark_true_false(df, 'WE', 'X')

In [141]:
df.head()

,EinkBeleg,Pos,Material,Werk,LOrt,Bestellmenge,BME,WEZ,ELi,WE,Anlegedatum
0,48175634,1,92181600,99,208T,3000,ST,0,True,True,2022-07-04
1,48175634,2,92182501,99,208T,2000,ST,0,True,True,2022-07-04
2,48175634,3,92182600,99,208T,2000,ST,0,True,True,2022-07-04
3,48175634,4,92219900,99,208T,5000,ST,0,True,True,2022-07-04
4,48175634,5,92194900,99,208T,11520,ST,0,True,True,2022-07-04


In [142]:
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\EKPO.csv', index=False)

### EKKO

In [152]:
df = pd.read_excel(xls, 'EKKO')

In [153]:
df.head()

,EinkBeleg,T,Art,Angel.am,Angel.von,Lieferant,BelegDat,LWk,IncTm
0,48175634,F,UB,2022-07-04,VARGAL,99005.0,2022-07-04,5.0,EXW
1,48175677,F,MPO,2022-07-05,MARKIEWICZJ,99010.0,2022-07-05,10.0,EXW
2,48175722,F,MPO,2022-07-06,MARKIEWICZJ,99010.0,2022-07-06,10.0,EXW
3,48175806,F,MPO,2022-07-12,MARKIEWICZJ,99010.0,2022-07-12,10.0,EXW
4,48175820,F,MPO,2022-07-12,MARKIEWICZJ,99010.0,2022-07-12,10.0,EXW


In [154]:
df[df['Angel.am'] != df['BelegDat']].shape

(0, 9)

In [155]:
df['Beleg_Anlege_Dat'] = df['Angel.am']
df.drop(['Angel.am', 'BelegDat'], axis=1, inplace=True)

In [156]:
df.head()

,EinkBeleg,T,Art,Angel.von,Lieferant,LWk,IncTm,Beleg_Anlege_Dat
0,48175634,F,UB,VARGAL,99005.0,5.0,EXW,2022-07-04
1,48175677,F,MPO,MARKIEWICZJ,99010.0,10.0,EXW,2022-07-05
2,48175722,F,MPO,MARKIEWICZJ,99010.0,10.0,EXW,2022-07-06
3,48175806,F,MPO,MARKIEWICZJ,99010.0,10.0,EXW,2022-07-12
4,48175820,F,MPO,MARKIEWICZJ,99010.0,10.0,EXW,2022-07-12


In [157]:
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\EKKO.csv', index=False)

### EKBE

In [170]:
df = pd.read_excel(xls, 'EKBE')

In [172]:
df = df[df['BTy'] == 'E']
df = df[(df['BwA'] == 101.) | (df['BwA'] == 109.)]

In [174]:
df.head()

,EinkBeleg,Pos,BTy,BwA,Buch.dat.,Menge
0,48175634,1,E,101.0,2022-07-04,3000
2,48175634,2,E,101.0,2022-07-04,2000
4,48175634,3,E,101.0,2022-07-04,2000
6,48175634,4,E,101.0,2022-07-04,5000
8,48175634,5,E,101.0,2022-07-04,11520


In [173]:
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\EKBE.csv', index=False)

### MVER

In [180]:
df = pd.read_excel(xls, 'MVER')

In [190]:
df.shape

(84, 15)

In [182]:
# Show unique values
df[2023].unique()

array(['Material', '560050R1P', '5614R001P', 61630001, 92218068, 92219900,
       92249068, '9M78S101', 'FG0603809', 'FP603476', 'TVS00035100000',
       'TVS10535311061', 'UBQ170OBE2V-01', nan, 2022], dtype=object)

In [248]:
df_1 = pd.read_excel(xls, 'MVER', nrows=74)
df_2 = pd.read_excel(xls, 'MVER', skiprows=76, nrows=160-76)

In [238]:
df_1

,Material,Werk,Jahr,Jan.23,Feb.23,Mrz.23,Apr.23,Mai.23,Jun.23,Jul.23,Aug.23,Sep.23,Okt.23,Nov.23,Dez.23
1,560050R1P,0010,2023,0,2,0,276,1014,298,294,0,1020,937,0,1020
2,560050R1P,0011,2023,435,1332,928,96,540,231,168,924,0,774,222,891
3,560050R1P,0013,2023,1650,1852,182,660,886,698,879,108,568,289,343,583
4,560050R1P,0028,2023,0,0,0,0,0,0,0,31,102,18,2,4
5,560050R1P,0042,2023,56,492,178,662,886,695,879,77,466,271,341,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,UBQ170OBE2V-01,0042,2023,25,132,49,33,247,548,51,429,45,45,36,24
69,UBQ170OBE2V-01,0058,2023,26,16,19,4,1,17,25,32,26,8,8,8
70,UBQ170OBE2V-01,0072,2023,435,736,559,760,784,553,1025,245,506,448,417,217
71,UBQ170OBE2V-01,0078,2023,443,612,532,634,624,577,607,440,439,435,392,446


In [237]:
df_2

,Material,Werk,Jahr,Jan.22,Feb.22,Mrz.22,Apr.22,Mai.22,Jun.22,Jul.22,Aug.22,Sep.22,Okt.22,Nov.22,Dez.22
1,560050R1P,0010,2022,115,12,44,689,1473,1061,732,1287,76,3,0,2
2,560050R1P,0011,2022,0,0,0,0,1,0,0,0,0,0,64,420
3,560050R1P,0013,2022,0,0,0,1984,961,48,0,0,12,420,1,432
4,560050R1P,0042,2022,502,66,733,71,530,762,603,1007,114,195,415,580
5,5614R001P,0005,2022,0,36,132,0,2,240,36,60,12,228,24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,UBQ170OBE2V-01,0042,2022,257,182,199,45,99,167,31,15,268,33,12,33
79,UBQ170OBE2V-01,0058,2022,2,41,52,32,58,51,2,0,12,24,16,3
80,UBQ170OBE2V-01,0072,2022,968,1360,788,664,448,671,481,0,698,247,654,296
81,UBQ170OBE2V-01,0078,2022,489,440,480,491,506,468,325,382,418,618,526,361


In [213]:
# append the title of the first column to the other columns
val = str(df_2.columns[0])[2:]
for title in df_2.columns[1:]:
    df_2.rename(columns={title: title + '.' + val}, inplace=True)

In [221]:
# change the name of the first three columns to the values of the first row
for i in range(3):
    df_2.rename(columns={df_2.columns[i]: df_2.iloc[0, i]}, inplace=True)

In [229]:
df_2.head()

,2022,Unnamed: 1.22,Unnamed: 2.22,Jan.22,Feb.22,Mrz.22,Apr.22,Mai.22,Jun.22,Jul.22,Aug.22,Sep.22,Okt.22,Nov.22,Dez.22
0,Material,Werk,Jahr,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.
1,560050R1P,0010,2022,115,12,44,689,1473,1061,732,1287,76,3,0,2
2,560050R1P,0011,2022,0,0,0,0,1,0,0,0,0,0,64,420
3,560050R1P,0013,2022,0,0,0,1984,961,48,0,0,12,420,1,432
4,560050R1P,0042,2022,502,66,733,71,530,762,603,1007,114,195,415,580


In [231]:
def change_month_to_monthYear(df: pd.DataFrame) -> pd.DataFrame:
    """
    Change the name of the columns to the format MonthYear
    """
    val = str(df.columns[0])[2:]
    for title in df.columns[1:]:
        df.rename(columns={title: title + '.' + val}, inplace=True)
    return df

def change_first_three_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Change the name of the first three columns to the values of the first row
    """
    for i in range(3):
        df.rename(columns={df.columns[i]: df.iloc[0, i]}, inplace=True)
    return df

In [249]:
df_2 = change_month_to_monthYear(df_2)
df_1 = change_month_to_monthYear(df_1)

df_2 = change_first_three_columns(df_2)
df_1 = change_first_three_columns(df_1)

In [250]:
df_2 = df_2.iloc[1:]
df_1 = df_1.iloc[1:]

df_1.drop(['Jahr'], axis=1, inplace=True)
df_2.drop(['Jahr'], axis=1, inplace=True)

In [264]:
# Join the two dataframes on Material
df = df_1.merge(df_2, on=['Material', 'Werk'], how='outer')

In [269]:
df.head(100)

,Material,Werk,Month,Absatz
0,61630001,0010,Jan.23,885
1,61630001,0011,Jan.23,602
2,61630001,0013,Jan.23,1463
3,61630001,0028,Jan.23,0
4,61630001,0041,Jan.23,0
...,...,...,...,...
95,61630001,0013,Feb.23,158
96,61630001,0028,Feb.23,0
97,61630001,0041,Feb.23,0
98,61630001,0042,Feb.23,655


In [266]:
df.fillna(0, inplace=True)

C:\Users\TimKnudsen\AppData\Local\Temp\ipykernel_6688\4231983114.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


In [268]:
df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='Absatz')

In [270]:
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\MVER.csv', index=False)

### DIM_Werk

In [19]:
df = pd.read_excel(xls, 'Absatz')

In [20]:
df = df.iloc[1:]

In [23]:
df.rename(columns={'Geschäftsj./Periode': 'Werk_Name'}, inplace=True)

In [24]:
dim_werk = df[['Werk', 'Werk_Name']].drop_duplicates()

In [25]:
dim_werk.head()

,Werk,Werk_Name
1,11.0,VILLEROY & BOCH AG
2,28.0,Villeroy&Boch Tradin
3,42.0,Villeroy&Boch Tradin
6,41.0,Villeroy & Boch OOO
8,49.0,Villeroy & Boch (Tha


In [27]:
dim_werk.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\DIM_werk.csv', index=False)

### Absatz

In [10]:
df = pd.read_excel(xls, 'Absatz')

In [11]:
df = df.iloc[1:]
df.drop(['Unnamed: 1', 'Geschäftsj./Periode', 'Gesamtergebnis'], axis=1, inplace=True)
df.fillna(0, inplace=True)

In [12]:
df = pd.melt(df, id_vars=['Artikel', 'Werk'], var_name='Month', value_name='Absatz')

In [13]:
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\Absatz.csv', index=False)

In [343]:
df.dtypes

Artikel     object
Werk       float64
Month       object
Absatz     float64
dtype: object

In [341]:
from datetime import datetime

In [345]:
df[df.Month.str.contains('MAI')]['Month']

308     MAI 2022
309     MAI 2022
310     MAI 2022
311     MAI 2022
312     MAI 2022
          ...   
1304    MAI 2023
1305    MAI 2023
1306    MAI 2023
1307    MAI 2023
1308    MAI 2023
Name: Month, Length: 154, dtype: object

In [336]:
df[df['Month'].str.contains('MAI')] = 'MAY' 

C:\Users\TimKnudsen\AppData\Local\Temp\ipykernel_6688\4208946664.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'MAY' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[df['Month'].str.contains('MAI')] = 'MAY'
C:\Users\TimKnudsen\AppData\Local\Temp\ipykernel_6688\4208946664.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'MAY' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[df['Month'].str.contains('MAI')] = 'MAY'


In [337]:
# Parse datetime from the Month column
df['date'] = df['Month'].apply(lambda x: datetime.strptime(x, '%b %Y'))

ValueError: time data 'MAY' does not match format '%b %Y'

### IO

In [392]:
def io_df(df: pd.DataFrame) -> pd.DataFrame:
    df.rename(columns={'Year': 'Werk'}, inplace=True)
    df.drop(['Unnamed: 25'], axis=1, inplace=True)
    df['Material'] = df[df.drop(['Werk'], axis=1).isnull().all(axis=1)]['Werk']
    df['Material'] = df['Material'].fillna(method='ffill')
    df = df[df['Werk'] != df['Material']]
    # Clear lines where 'Total' appears in the Werk column
    df = df[df['Werk'].str.contains('Total') == False]  
    df = df[df['Werk'].str.contains('TOTAL') == False]
    df.fillna(0, inplace=True)
    df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='value')
    df['Material'] = df['Material'].str.split().str[1]
    df['Werk'] = df['Werk'].str.split().str[0]
    df['Werk'] = df['Werk'].astype(int)
    df['Month'] = df['Month'].str[-6:]
    return df

In [397]:
df = pd.read_excel(xls, 'IO', header=1)

In [399]:
df = io_df(df)

C:\Users\TimKnudsen\AppData\Local\Temp\ipykernel_6688\266850261.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Material'] = df['Material'].fillna(method='ffill')


In [372]:
df.rename(columns={'Year': 'Werk'}, inplace=True)
df.drop(['Unnamed: 25'], axis=1, inplace=True)
df['Material'] = df[df.drop(['Werk'], axis=1).isnull().all(axis=1)]['Werk']

In [400]:
df

,Material,Werk,Month,value
0,560050R1P,11,Jan.22,0.0
1,560050R1P,28,Jan.22,0.0
2,560050R1P,42,Jan.22,546.0
3,61630001,11,Jan.22,345.0
4,61630001,28,Jan.22,0.0
...,...,...,...,...
1843,UBQ170OBE2V-01,42,Dec.23,18.0
1844,UBQ170OBE2V-01,58,Dec.23,16.0
1845,UBQ170OBE2V-01,78,Dec.23,316.0
1846,UBQ170OBE2V-01,99,Dec.23,0.0


In [378]:
# Use only the ID in Material
df['Material'] = df['Material'].str.split().str[1]

In [380]:
df['Werk'] = df['Werk'].str.split().str[0]

In [382]:
df['Month'] = df['Month'].str[-6:]

In [32]:
from preprocessing import io_df, bestelluebersicht_df, material_dim_df

xls = pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung.xlsx')

In [33]:
df = pd.read_excel(xls, 'IO', header=1)
df = io_df(df)
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\IO.csv', index=False)

c:\celver\scm_optimization\v_b\code\preprocessing.py:154: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df[df['Werk'].str.contains('TOTAL') == False]


In [35]:
df = pd.read_excel(xls, 'Bestellübersicht')
df = bestelluebersicht_df(df)
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\Bestellübersicht.csv', index=False)

In [37]:
df = pd.read_excel(xls, 'Bestellübersicht')
df = material_dim_df(df)
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\DIM_Material.csv', index=False)

#### AFKO und AUFK

In [38]:
df = pd.read_excel(xls, 'AFKO')
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\AFKO.csv', index=False)

In [40]:
df = pd.read_excel(xls, 'AUFK')
df.to_csv('c:\\celver\\scm_optimization\\v_b\\preprocessed_data\\AUFK.csv', index=False)

In [41]:
df.head()

,Auftrag,Erf.datum,Werk
0,1501288299,2022-05-16,54
1,1501294971,2022-06-07,54
2,1501294972,2022-06-07,54
3,1501304289,2022-07-14,54
4,1501313001,2022-08-22,54


# DIM_Artikel

In [2]:
import pandas as pd

In [3]:
xls = pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')

In [22]:
df = pd.read_excel(xls, 'BestÜ neu')

In [23]:
# Get unique material numbers and Kurztext
dim_material = df.groupby('Material')['Kurztext'].apply(lambda x: x.iloc[0]).reset_index()
dim_material

,Material,Kurztext
0,560050R1P,Subway 2.0 WWC for ViClean packed
1,5614R001P,Subway 2.0 WWC DirectFlush verpackt
2,61630001,Loop & Friends Unterbau-WB 540x345mm
3,92218068,ViConnect 300S Betätigungs Pl. dual fl
4,92219900,Fixation Set WWC Suprafix 3.0
5,92249068,ViConnect 200S Betätigungs Pl. dual fl
6,9M78S101,Subway 2.0 WC Sitz softcl/quick-rel.
7,TVS00035100000,Univ. Vi-Box
8,TVS10535311061,O.novo Start conc. mixer
9,UBQ170OBE2V-01,Oberon bathtub 170x75 rect


In [24]:
df = pd.read_excel(xls, 'AFKO neu')
# get unique Material numbers
dim_material_2 = df[['Material']].drop_duplicates().reset_index(drop=True)
dim_material_2

,Material
0,FP603476
1,FG0603809


In [25]:
concat_dim_material = pd.concat([dim_material, dim_material_2], ignore_index=True)
concat_dim_material

,Material,Kurztext
0,560050R1P,Subway 2.0 WWC for ViClean packed
1,5614R001P,Subway 2.0 WWC DirectFlush verpackt
2,61630001,Loop & Friends Unterbau-WB 540x345mm
3,92218068,ViConnect 300S Betätigungs Pl. dual fl
4,92219900,Fixation Set WWC Suprafix 3.0
5,92249068,ViConnect 200S Betätigungs Pl. dual fl
6,9M78S101,Subway 2.0 WC Sitz softcl/quick-rel.
7,TVS00035100000,Univ. Vi-Box
8,TVS10535311061,O.novo Start conc. mixer
9,UBQ170OBE2V-01,Oberon bathtub 170x75 rect


In [28]:
concat_dim_material.to_parquet('c:\\celver\\scm_optimization\\v_b\\new_preprocessed\\DIM_Material.parquet', index=False)